# Question 2 (50 points)
In this question, you will simulate a peptide design experiment, trying to find peptides with high binding affinity to MHC class I using a bayesian optimization approach. Notice the goal here is not trying to find a peptide sequence that maximize the binding affinity to MHC, Since a sizable proportion of the sequence data we are using contains maximum binding affinity out of the data (9.0). Using the same feature encoding as question 1, we will examine several techniques to maximize the percentage of sequence with affinity of 9.0 for stringent querying.

$\textbf{Complete the code under ###TO DO in each cell and produce the required plots.}$ Feel free to define any helper functions as you see fit.

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import shuffle

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.ensemble import RandomForestRegressor

from modAL.models import BayesianOptimizer
from modAL.acquisition import max_EI

import seqlogo

import copy

### Set randome seed
seed = 5
random.seed(seed)
np.random.seed(seed)

# Reading and Processing the Data¶

In [ ]:
data = np.loadtxt('hw3_data.csv', dtype = str, delimiter = ',')[1:]
### TO DO

# 2.1: Random Sampling (5 pts. total) 

Create a random query strategy for randomly selecting a sample to query from the data. If the data selected is a new sequence with binding affnity of 9.0, append it to a list. After each query selection, measure the percentage of sequence with binding affnity 9.0 found by the strategy. Do this for 200 sampling steps. This will serve as the baseline to compare with optimizator performance in section 2.2 and 2.3.

In [ ]:
X_cp = copy.deepcopy(X)
y_cp = copy.deepcopy(y)
optimal_idx_rand = []
history_rand = [0]

### TO DO

# 2.2: Baysian Optimization with Gaussian Process (15 pts. total)

Create a Baysian optimizer with Gaussian process as regressor and Max Expected improvement as the queuing strategy. If the data selected is a new sequence with binding affinity of 9.0, append it to a list. After each query selection, measure the percentage of sequence with binding affinity 9.0 found by the strategy. Do this for 200 sampling steps. 

$\textbf{Hint}$: Check the <a href="https://modal-python.readthedocs.io/en/latest/content/examples/bayesian_optimization.html#Optimizing-using-expected-improvement"> modAL documentation</a> for how to set up a Baysian optimizer.

In [ ]:
X_cp = copy.deepcopy(X)
y_cp = copy.deepcopy(y)

optimal_idx_gp = []

### TO DO

# 2.3: Bayesian Optimizer with Random Forest (10 pts. total)

Although Baysian optimization often uses the Gaussian process, Baysian optimizer in ModAL can take any other regressor that has a predict function with a return_std input parameter. If return_std is set to True, the function returns the predicted values and standard deviation in the prediction. Create a Baysian optimizer with random forest regressor and Max Expected improvement as the queuing strategy. If the data selected is a new sequence with binding affinity of 9.0, append it to a list. After each query selection, measure the percentage of sequences with binding affinity 9.0 found by the strategy. Do this for 200 sampling steps. 

$\textbf{Hint}$: You might find the following class wrapper for random forest helpful.

In [ ]:
class rfwapper(RandomForestRegressor):
    def predict(self, X, return_std = False):
        if return_std:
            ys = np.array([e.predict(X) for e in self.estimators_])
            return np.mean(ys, axis = 0).ravel(), np.std(ys, axis = 0).ravel()
        return super().predict(X).ravel()

In [ ]:
X_cp = copy.deepcopy(X)
y_cp = copy.deepcopy(y)

optimal_idx_rf = []

### TO DO

# 2.4: Plot Percentage of sequence with maximum binding affinity with respect to number of sequence queried (10 pts. total)

In [ ]:
# Plot our performance over time.
fig, ax = plt.subplots(figsize=(6, 6), dpi=130)

ax.plot(history)
ax.scatter(range(len(history)), history, s=13, label = 'Baysian')


ax.plot(history_rf)
ax.scatter(range(len(history_rf)), history_rf, s=13, label = 'RF')


ax.plot(history_rand)
ax.scatter(range(len(history_rand)), history_rand, s=13, label = 'Random')

ax.grid(True)

ax.set_title('Incremental classification accuracy')
ax.set_xlabel('Query iteration')
ax.set_ylabel('Percentage of sequence with pIC50 = 9.0')
ax.legend()
plt.show()

# 2.5: Create sequence logo based on sequences found with each querying strategies (5 pts. total)

A sequence logo is a graphical representation of the sequence conservation of amino acids in protein sequences), as amino acids that are important for functions are likely to be conserved. Hence, a sequence logo is a way to visualize such an importance. Convert the each sets of sequences obtained by one of your optimization strategies to a sequence logo. Below is an example using all of the sequence of affinity 9.0.


$\textbf{Important}$: We are using <a href="https://pypi.org/project/seqlogo/"> seqlogo</a> to create sequence logo from our set of sequences. You can install seqlogo by entering the command 

conda install -c bioconda seqlogo 

in your conda terminal

In [ ]:
X_opt = X[np.where(y == 9)[0]]
ppm = np.sum(X_opt, axis = 0).reshape(20,9)
ppm /= np.sum(ppm, axis = 0)
ppm = seqlogo.Ppm(ppm, alphabet_type = 'AA')
seqlogo.seqlogo(ppm, ic_scale = False, format = 'jpeg', size = 'medium')

In [ ]:
### TO DO